## Plot markers umap embeddings
Does this over each donor and separate

In [1]:
# Input info
se_f = "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_prefilterMerge_impute/knn/kparam_30/gff_A2_black/annotation_clones/SE.rds"
#sample_names = "preB,postB"
outdir = "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_prefilterMerge_impute/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap"


# Parallel processing
cores = 4

# R utils script
utils_script = "/data/Mito_Trace/R_scripts/seurat_utils.R"
markers_f = "/data/Mito_Trace/data/processed/cd34_markers_v02.txt"



In [2]:
library("Nebulosa")

Loading required package: ggplot2

Warning message:
“package ‘ggplot2’ was built under R version 4.1.3”
Loading required package: patchwork

Registered S3 method overwritten by 'spatstat.core':
  method          from
  formula.glmmPQL MASS



In [3]:
source(utils_script)
#outdir = outdir
dir.create(outdir, showWarnings = FALSE)


Attaching SeuratObject

Attaching sp

Warning message:
“package ‘dplyr’ was built under R version 4.1.3”

Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [4]:
# sample_names <- unlist(strsplit(sample_names, ",")[[1]])
# sample_names

In [5]:
library(GenomicRanges)
library(Seurat)
library(Signac)
library(GenomeInfoDb)
library(EnsDb.Hsapiens.v75)
library(ggplot2)
library(patchwork)
set.seed(1234)
library(data.table)
library(magrittr)
library(cowplot)
library(metap)
library("RColorBrewer")

library(future)
plan()

plan("multiprocess", workers = cores)
#options(future.globals.maxSize = 50000 * 1024^2) # for 50 Gb RAM
options(future.globals.maxSize = 8000 * 1024^2)

library(dplyr)

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors

Warning message:
“package ‘S4Vectors’ was built under R version 4.1.2”

Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:dplyr’:

    first, rename


The following objects are masked from 

sequential:
- args: function (..., envir = parent.frame())
- tweaked: FALSE
- call: NULL

Warning message:
"Strategy 'multiprocess' is deprecated in future (>= 1.20.0). Instead, explicitly specify either 'multisession' or 'multicore'. In the current R session, 'multiprocess' equals 'multicore'."


In [6]:
se <- readRDS(se_f)#file.path(indir, paste0("allSamples.integrated.rds")))
se

DefaultAssay(se) <- "RNA"

An object of class Seurat 
212927 features across 13593 samples within 2 assays 
Active assay: RNA (19039 features, 0 variable features)
 1 other assay present: ATAC
 2 dimensional reductions calculated: integrated_lsi, umap

In [7]:
markers <- read.csv(markers_f, header=F)[[1]]

In [8]:
markers


[1] "HLF"     "CD34"    "CD38"    "PTPRC"   "THY1"    "ITGA6"   "PF4"    
 [8] "CA1"     "HBA2"    "LY86"    "LYZ"     "GSTM1"   "FCN1"    "LTF"    
[15] "TPSAB1"  "PRG2"    "CD74"    "Siglec9" "VPREB1"  "MYL4"    "FCRLA"  
[22] "KIT"     "ADGRG1"  "DNTT"    "FLT3"    "LY6D"    "CD7"     "ITGB7"  
[29] "NAAA"    "CSF1R"   "F13A1"   "ELANE"   "IRF8"    "CEPBA"   "SPI1"   
[36] "SATB1"   "CD33"    "CD14"    "CD4"     "CD19"

## Plot marker embeddings

In [9]:
# featplot <- function(name.sig, se, curr.outdir, feat.names=NULL){
#   if (!is.null(feat.names)){
#     name <- feat.names[name.sig,]
#   }else{
#     name <- name.sig
#   }
#   if (name.sig %in% rownames(se)){
#     feat <- FeaturePlot(se,  features=name.sig) + ggtitle(name)
#     ggsave(plot=feat,
#            file=file.path(curr.outdir, paste0(name.sig,".embedFeat.top.png")))
#   }else{
#     print(paste0("Feature no in object: ", name.sig))
#   }

In [10]:
featplot <- function(name.sig, se, curr.outdir, use.density){
    if (name.sig %in% rownames(se)){
        if(!file.exists(file.path(curr.outdir, paste0(name.sig,".embedFeat.top.png")))){
            feat <- FeaturePlot(se,  features=name.sig)
            ggsave(plot=feat,
                   file=file.path(curr.outdir, paste0(name.sig,".embedFeat.top.png")))
            ggsave(plot=feat,
                   file=file.path(curr.outdir, paste0(name.sig,".embedFeat.top.svg")))

            if(use.density){
                print("plot density")
                f <- plot_density(se, name.sig) + ggtitle(name.sig)
                ggsave(plot=f,
                   file=file.path(curr.outdir, paste0("density.", name.sig,".embedFeat.top.png")))
                ggsave(plot=f,
                   file=file.path(curr.outdir, paste0("density.", name.sig,".embedFeat.top.svg")))
            }
        }else{print("file already here. not overwriting. Please delete first")}
    }
}


sapply(markers, FUN=featplot, se=se, curr.outdir=outdir, use.density=TRUE)

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



[1] "plot density"


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



[1] "plot density"


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



...

[1] "plot density"


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



[1] "plot density"


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



$HLF
[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_prefilterMerge_impute/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.HLF.embedFeat.top.svg"

$CD34
[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_prefilterMerge_impute/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.CD34.embedFeat.top.svg"

$CD38
[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_prefilterMerge_impute/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.CD38.embedFeat.top.svg"

$PTPRC
[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_prefilterMerge_impute/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.PTPRC.embedFeat.top.svg"

$THY1
[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_prefilterMerge_impute/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.THY1.embedFeat.top.svg"

$ITGA6
[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_prefilterMerge_impute/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.ITGA6.embedFeat.top.svg"

$PF4
[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_prefilterMerge_impute/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.PF4.embedFeat.top.svg"

$CA1
[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_prefilterMerge_impute/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.CA1.embedFeat.top.svg"

$HBA2
[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_prefilterMerge_impute/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.HBA2.embedFeat.top.svg"

$LY86
[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_prefilterMerge_impute/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.LY86.embedFeat.top.svg"

$LYZ
[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_prefilterMerge_impute/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.LYZ.embedFeat.top.svg"

$GSTM1
[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_prefilterMerge_impute/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.GSTM1.embedFeat.top.svg"

$FCN1
[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_prefilterMerge_impute/knn/kparam_30/gff_A2_blac

## Plot marker embeddings with peaks

In [11]:
DefaultAssay(se) <- "ATAC"
feature.names <- ClosestFeature(se, rownames(se))


In [12]:
featplot <- function(name.sig, se, curr.outdir, feat.names=NULL, use.density=TRUE){
    if (!is.null(feat.names)){
        name <- feat.names[name.sig,]
    }else{
        name <- name.sig
    }
    if (name.sig %in% rownames(se)){
        feat <- FeaturePlot(se,  features=name.sig) + ggtitle(name)
        ggsave(plot=feat,
               file=file.path(curr.outdir, paste0(name.sig,".embedFeat.top.png")))  
        ggsave(plot=feat,
               file=file.path(curr.outdir, paste0(name.sig,".embedFeat.top.svg")))  
        if(use.density){
            f <- plot_density(se, name.sig) + ggtitle(name)
            ggsave(plot=f,
                   file=file.path(curr.outdir, paste0("density.", name.sig,".embedFeat.top.png")))
            ggsave(plot=f,
                   file=file.path(curr.outdir, paste0("density.", name.sig,".embedFeat.top.svg")))
        }
    }
}

In [13]:
marker.feature.names <- feature.names %>% dplyr::filter(gene_name %in% markers)
rownames(marker.feature.names) <- marker.feature.names[,"query_region"]
marker.feature.names

marker.feature.names$name <- paste0(marker.feature.names$gene_name, "_", marker.feature.names$type)



sapply(rownames(marker.feature.names), 
       FUN=featplot, se=se, curr.outdir=outdir, feat.names=marker.feature.names["name"])

,source,type,score,phase,gene_id,gene_version,gene_type,gene_name,level,hgnc_id,⋯,exon_number,exon_id,exon_version,protein_id,ccdsid,ont,gene_biotype,closest_region,query_region,distance
,<fct>,<fct>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
chr1-109687156-109688838,HAVANA,UTR,NA,NA,ENSG00000134184,13,protein_coding,GSTM1,1,HGNC:4632,⋯,1,ENSE00001450987,1,ENSP00000358834.2,NA,NA,protein_coding,chr1-109687858-109687873,chr1-109687156-109688838,0
chr1-109703213-109703518,HAVANA,transcript,NA,NA,ENSG00000134184,13,protein_coding,GSTM1,1,HGNC:4632,⋯,NA,NA,NA,ENSP00000455929.1,NA,NA,protein_coding,chr1-109687845-109708685,chr1-109703213-109703518,0
chr1-161706673-161707493,HAVANA,UTR,NA,NA,ENSG00000132185,16,protein_coding,FCRLA,2,HGNC:18504,⋯,1,ENSE00001445991,1,ENSP00000356927.1,NA,NA,protein_coding,chr1-161707295-161707333,chr1-161706673-161707493,0
chr1-198620169-198622958,HAVANA,gene,NA,NA,ENSG00000081237,20,protein_coding,PTPRC,2,HGNC:9666,⋯,NA,NA,NA,NA,NA,NA,protein_coding,chr1-198638457-198757476,chr1-198620169-198622958,15498
chr1-198623943-198624401,HAVANA,gene,NA,NA,ENSG00000081237,20,protein_coding,PTPRC,2,HGNC:9666,⋯,NA,NA,NA,NA,NA,NA,protein_coding,chr1-198638457-198757476,chr1-198623943-198624401,14055
chr1-198633499-198634273,HAVANA,gene,NA,NA,ENSG00000081237,20,protein_coding,PTPRC,2,HGNC:9666,⋯,NA,NA,NA,NA,NA,NA,protein_coding,chr1-198638457-198757476,chr1-198633499-198634273,4183
chr1-198638192-198644274,HAVANA,exon,NA,NA,ENSG00000081237,20,protein_coding,PTPRC,2,HGNC:9666,⋯,1,ENSE00002165642,1,NA,NA,NA,protein_coding,chr1-198639288-198639334,chr1-198638192-198644274,0
chr1-198645150-198646114,HAVANA,transcript,NA,NA,ENSG00000081237,20,protein_coding,PTPRC,2,HGNC:9666,⋯,NA,NA,NA,NA,NA,NA,protein_coding,chr1-198639288-198703346,chr1-198645150-198646114,0
chr1-198646687-198647455,HAVANA,transcript,NA,NA,ENSG00000081237,20,protein_coding,PTPRC,2,HGNC:9666,⋯,NA,NA,NA,NA,NA,NA,protein_coding,chr1-198639288-198703346,chr1-198646687-198647455,0


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 

chr1-109687156-109688838 
 "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_prefilterMerge_impute/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.chr1-109687156-109688838.embedFeat.top.svg" 
                                                                                                                                                                                                                                                                                                          chr1-109703213-109703518 
 "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_prefilterMerge_impute/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.chr1-109703213-109703518.embedFeat.top.svg" 
                                                                                                                                                                                                                                                                                                          chr1-161706673-161707493 
 "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_prefilterMerge_impute/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.chr1-161706673-161707493.embedFeat.top.svg" 
                                                                                                                                                                                                                                                                                                          chr1-198620169-198622958 
 "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_prefilterMerge_impute/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.chr1-198620169-198622958.embedFeat.top.svg" 
                                                                                                                                                                                                                                                                                                          chr1-198623943-198624401 
 "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_prefilterMerge_impute/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.chr1-198623943-198624401.embedFeat.top.svg" 
                                                                                                                                                                                                                                                                                                          chr1-198633499-198634273 
 "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_prefilterMerge_impute/knn/kparam_30/gff_A2_black/annotation_clones/markers/scrap/density.chr1-198633499-198634273.embedFeat.top.svg" 
                                                                                                                                                                                                                                                                                                          chr1-198638192-198644274 
 "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_Tru

---

---

In [14]:
sessionInfo()

R version 4.1.1 (2021-08-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 20.04.4 LTS

Matrix products: default
BLAS/LAPACK: /home/isaac/miniconda3/envs/mttrace/lib/libopenblasp-r0.3.20.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] future_1.25.0             RColorBrewer_1.1-3       
 [3] metap_1.1                 cowplot_1.1.1            
 [5] magrittr_2.0.3            data.table_1.14.2        
 [7] EnsDb.Hsapiens.v75_2.99.0 ensembldb_2.18.1         
 [9] AnnotationFilter_1.18.0   GenomicFeatures_1.46.1   
[11] 